In [16]:
import numpy as np
import torch
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from model_PCA_correlation import AttentionModel_PCA
from dcascore import *
from utils import read_fasta_alignment, remove_duplicate_sequences, add_PCA_coords

# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from model import AttentionModel
from gillespie_main import generate_gill_n_save
from seq_utils import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq 


In [13]:
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]


In [14]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens.shape)

torch.Size([64, 65, 65])
35
65
torch.Size([35, 35, 65, 65])


In [5]:

save_dir = "gill_generated_sequences_brute_force"
N_seqs = 30000
save_name = "gill_generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))
N_seqs=8000
betas = [ 0.1, 0.5, 1, 2, 4, 10]

for b in betas:
    save_name = f"gill_gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1_PCA_comp_24_20"
    #generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,PCA_comp_list=np.array([24,20]))
    if b!=1:
        save_name = f"gill_gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}_PCA_comp_24_20"
        generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=nb_PCA_comp,beta_PCA=b,PCA_comp_list=np.array([24,20]))

"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp)
N_seqs=8000
betas = [ 0.1, 0.5, 1, 2, 4, 10]
for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"
    #generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num.copy(),nb_PCA_comp=nb_PCA_comp, beta=b)
    if b!=1:
        save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b_1_b_PCA{b}"
        generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num.copy(), beta=1,nb_PCA_comp=nb_PCA_comp,beta_PCA=b)

100%|██████████| 8000/8000 [14:21<00:00,  9.28it/s]


Generated sequences (letters): ['-PFAKSWNR-GCYRFFFIIRD-FHEPQTQGCGAVP-ERTMHLPQVKDCTDGRRMKHAGQQQ-A', '-PFAKSWNR-GCYRFFFIIRD-FHEPQTQGCGAVP-ERTMHLPQVKDCTDGRRMKHAGQQR-A', '-PFAKSWNR-GCYRFFFIIRD-FHEPQTQGMGAVP-ERTMHLPQVKDCTDGRRMKHAGQQR-A', '-PFAKSWNR-GCYRFFFIIRD-FHEPQTQGMLAVP-ERTMHLPQVKDCTDGRRMKHAGQQR-A', '-PFAKSWNR-GCYRFFFIIRD-FHEPQTQGMQAVP-ERTMHLPQVKDCTDGRRMKHAGQQR-A']
Generated sequences saved to gill_generated_sequences_brute_forcePCA_comp


100%|██████████| 8000/8000 [14:18<00:00,  9.31it/s]


Generated sequences (letters): ['R-KWLYFYGHTMCEDQDFSNPGKDSIERCRYQSVFRTFNIPKDIHFIWCQPVMTVDITLYCLM', 'R-KWLYFYGHTMCEDQDFSNPGKDSIERCRYQSVFRTFNIPKDIHFIWPQPVMTVDITLYCLM', 'R-KWLYFYGHTMCEDQDFSNPGKDSIERCRYQSVFSTFNIPKDIHFIWPQPVMTVDITLYCLM', 'R-KWLYFYGHTMCEDQDFSNPGKDSIERCRYQSVFSTFNIPKDIHFAWPQPVMTVDITLYCLM', 'R-KWLYFYGHTMCEDQDFSNPGKDSIERCRYQSVFSTFNCPKDIHFAWPQPVMTVDITLYCLM']
Generated sequences saved to gill_generated_sequences_brute_forcePCA_comp


100%|██████████| 8000/8000 [14:18<00:00,  9.32it/s]


...

Generated sequences (letters): ['DPYEVLGI-EGASQAEIKKAYREFASKYDSDQYGDIPLAECKMRELSPEKKYNSWNV-T---N', 'DPYEVLGI-EGASQAEIKKAYREFASKYDSDQYGDIPLAECKMRELSPEKK-NSWNV-T---N', 'DPYEVLGI-EGASQAEIKKAYREFASKYDSDLYGDIPLAECKMRELSPEKK-NSWNV-T---N', 'DPYEVLGI-EGASQAEIKKAYREFASKYDSDLYVDIPLAECKMRELSPEKK-NSWNV-T---N', 'DPYEVLGI-EGASQNEIKKAYREFASKYDSDLYVDIPLAECKMRELSPEKK-NSWNV-T---N']
Generated sequences saved to gill_generated_sequences_brute_forcePCA_comp


100%|██████████| 8000/8000 [14:15<00:00,  9.36it/s]


Generated sequences (letters): ['DPYEVLGI-EGASQPEIKKAYREFASKYDSDQYGDRPLAECKMRELSPEKKYNSWNV-T---N', 'DPYEVLGI-EGASQPEIKKAYREFASKYDSDQYGDRPLAECKMRELSPENKYNSWNV-T---N', 'DPYEVLGI-EGASQIEIKKAYREFASKYDSDQYGDRPLAECKMRELSPENKYNSWNV-T---N', 'DPYEVLGI-EGASQIEIKKAYREFASKYDSDQYGDRPQAECKMRELSPENKYNSWNV-T---N', 'DPYEVLGI-EGASQIEIKKAYREFASKYDSDQYGDRPQAECKMKELSPENKYNSWNV-T---N']
Generated sequences saved to gill_generated_sequences_brute_forcePCA_comp


100%|██████████| 8000/8000 [14:14<00:00,  9.36it/s]


Generated sequences (letters): ['DPYEVLGI-EGASQPEIKKAYREFASKYDSDQYGDIILAECKMRELSPEKKYNSWNV-T---N', 'DPYEVLGI-EGASQPEIKKAYREFASKYDSDQYGDIELAECKMRELSPEKKYNSWNV-T---N', 'DPYEVLGI-EGASQPEIKKAYREFASKYDSDQYGDIELAECKMRELSPEEKYNSWNV-T---N', 'DPYEVLGI-EGASQAEIKKAYREFASKYDSDQYGDIELAECKMRELSPEEKYNSWNV-T---N', 'DPYEVLGISEGASQAEIKKAYREFASKYDSDQYGDIELAECKMRELSPEEKYNSWNV-T---N']
Generated sequences saved to gill_generated_sequences_brute_forcePCA_comp


In [15]:
save_dir = "gill_generated_sequences_brute_force_specific"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))
N_seqs=8000
betas = [8,12,16,20]
targey_list=[[24,20], [27,5],[8,8]]
for target in targey_list:
    for b in betas:
        save_name = f"gill_gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1_PCA_comp_{target[0]}_{target[1]}"
        #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,PCA_comp_list=np.array([24,20]))
        if b!=1:
            save_name = f"gill_gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}_PCA_comp_{target[0]}_{target[1]}"
            generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=nb_PCA_comp,beta_PCA=b,PCA_comp_list=np.array(target))


100%|██████████| 8000/8000 [19:33<00:00,  6.82it/s]


Generated sequences (letters): ['PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VSFLMCCFL', 'PSAEEILYWFLPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VSFLMCCFL', 'PSAEEILYWFLPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAMFGHK-VSFLMCCFL', 'PSAEEILYWFLPCHIRVGQKH-YTYGSSAWEVYYYRIACLAM-ENWDAMFGHK-VSFLMCCFL', 'PSAEEILYWFLPCHIRVGQKH-YTYGSSAWEVYYYRIACLGM-ENWDAMFGHK-VSFLMCCFL']
Generated sequences saved to gill_generated_sequences_brute_force_specificPCA_comp


100%|██████████| 8000/8000 [14:17<00:00,  9.33it/s]


Generated sequences (letters): ['--TRWI-EPNLDVHVHQY-YHFCSLTEVQFCMACWDQEYWMWKFPVYHLMEL-KWVGRFVGVY', '--TRWN-EPNLDVHVHQY-YHFCSLTEVQFCMACWDQEYWMWKFPVYHLMEL-KWVGRFVGVY', '--TRWN-EPNLDVHVHQY-YHFCSLTEVQFCMACWDQEYWMWKFPVGHLMEL-KWVGRFVGVY', '--TRWN-EPNLDVHVHQY-YHFCSLTEVSFCMACWDQEYWMWKFPVGHLMEL-KWVGRFVGVY', '--TRWN-EPNLDVHVHQYGYHFCSLTEVSFCMACWDQEYWMWKFPVGHLMEL-KWVGRFVGVY']
Generated sequences saved to gill_generated_sequences_brute_force_specificPCA_comp


100%|██████████| 8000/8000 [14:28<00:00,  9.21it/s]


...

Generated sequences (letters): ['FMMPMDNAFQGIYINDFPMEYSYRFDIRMSHGTSIYDHWWNCF-YQA-SWQ-RMELLDTATLW', 'FMMPMDNAFQGI-INDFPMEYSYRFDIRMSHGTSIYDHWWNCF-YQA-SWQ-RMELLDTATLW', 'FMMPMDNAFQGI-INDFPMEYSYRFDIRMS-GTSIYDHWWNCF-YQA-SWQ-RMELLDTATLW', 'FMMPMWNAFQGI-INDFPMEYSYRFDIRMS-GTSIYDHWWNCF-YQA-SWQ-RMELLDTATLW', 'FMMPMWNAFQGI-INDFPMEYSYRFDIRMS-GTSIYDHWWNCF-YQA-SWQ-RMELLDTAALW']
Generated sequences saved to gill_generated_sequences_brute_force_specificPCA_comp


100%|██████████| 8000/8000 [13:55<00:00,  9.57it/s]


Generated sequences (letters): ['NINGN-GSWGLPYFQPQGDMLTENHRSRPEYAIT-PFKVAWADCCKRAHHYSYVERK-GKEDH', 'NINGN-GSWGLPYFQPQGDMLTENHRSRPEYAIT-PFKVAWADCCKRAHHYSYVERK-GK-DH', 'NINGN-GSWGLPYFQPQGDMLTENHRSRPEYAIT-PFKVAWASCCKRAHHYSYVERK-GK-DH', 'NINGN-GSWGLPYMQPQGDMLTENHRSRPEYAIT-PFKVAWASCCKRAHHYSYVERK-GK-DH', 'NIIGN-GSWGLPYMQPQGDMLTENHRSRPEYAIT-PFKVAWASCCKRAHHYSYVERK-GK-DH']
Generated sequences saved to gill_generated_sequences_brute_force_specificPCA_comp


100%|██████████| 8000/8000 [13:55<00:00,  9.58it/s]

Generated sequences (letters): ['-CVL-KQQAK-VMGPAMQRPHMRKRMNIMCSP-ISHTEFIGIILHVNQWELFSFICDFDVTYS', '-CVL-KQQAK-VMGPAMQRPHMRKRMNIMCSP-ISHTEFIGIILHVNQWELFSFICDFDVTYL', '-CVL-KQQAK-VMGPAMQRPHMRKRMNIMCSP-ISHTEFIGIILHINQWELFSFICDFDVTYL', '-CVL-KQQAK-VMGPAMQRPHMRKRMNIMCSP-I-HTEFIGIILHINQWELFSFICDFDVTYL', '-CVL-KQQAK-VMDPAMQRPHMRKRMNIMCSP-I-HTEFIGIILHINQWELFSFICDFDVTYL']
Generated sequences saved to gill_generated_sequences_brute_force_specificPCA_comp


Model2

In [6]:
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

In [7]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens.shape)
print(Jtens.shape[-1])

torch.Size([64, 63, 63])
21
63
torch.Size([21, 21, 63, 63])
63


In [8]:
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N1=Q_1.shape
_,_,N2=K_1.shape
_,q1,q2=V_1.shape
model=AttentionModel_PCA(H,d,N1,N2,q1,q2,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

# i_indices = torch.arange(L, device=device).unsqueeze(1)
# j_indices = torch.arange(L, device=device).unsqueeze(0)
# mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
# W = W * mask
    
# Compute Jtens
Jtens_PCA = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens_PCA.shape)

torch.Size([64, 63, 2])
21
63
torch.Size([21, 35, 63, 2])


In [9]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "gill_generated_sequences_2Model_PCA"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]),J_PCA=Jtens_PCA)
N_seqs=8000
betas = [0.5, 1, 4, 10]

for b in betas:
    save_name = f"gill_gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1"
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA,PCA_comp_list=np.array([23,23]))
    if b!=1:
        save_name = f"gill_gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}"
        generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA,beta_PCA=b,PCA_comp_list=np.array([23,23]))

##############################################################

"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA)

"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4,10]

for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA)
    if b!=1:
        save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b_1_b_PCA{b}"
        #generate_gill_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=1,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA,beta_PCA=b)

AttributeError: 'SequenceGill' object has no attribute 'L'